In [62]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import math
from sklearn.metrics import mean_absolute_error


In [68]:
dataOriginal=pd.read_csv('./ml-100k/u.data',sep=r'\t',names=['user id','item id', 'rating', 'timestamp'])
filmesId=pd.DataFrame((set(dataOriginal['item id'])),columns=['Filmes'])

itemData=pd.read_csv('./ml-100k/u.item',sep=r'|',names=['movie_id ', 'movie title' , 'release date' ,' video release date' ,
              'IMDb URL' ,' unknown' , 'Action' , 'Adventure' , 'Animation' ,
             ' Childrens ', 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
              'Film-Noir' , 'Horror' , 'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi' ,
              'Thriller' , 'War' , 'Western' ], encoding='latin-1')
userData=pd.read_csv('./ml-100k/u.user',sep='|',names=['user id','age','Genero','Ocupacao','ZipCode'])

C:\Users\User\AppData\Local\Temp\ipykernel_7016\1420820392.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dataOriginal=pd.read_csv('./ml-100k/u.data',sep=r'\t',names=['user id','item id', 'rating', 'timestamp'])


In [81]:


def cossine_similarity(u, v,tabelaSimilaridade):
    u=u.reset_index()
    v=v.reset_index()
    itemsComuns = set(u['item id']).intersection(set(v['item id']))
    id_u=u['user id'][0]
    id_v = v['user id'][0]
    u = u[u['item id'].isin(itemsComuns)].sort_values(by='item id')['rating']
    v = v[v['item id'].isin(itemsComuns)].sort_values(by='item id')['rating']
    result = np.dot(u, v)/(np.linalg.norm(u)*np.linalg.norm(v))
    tabelaSimilaridade[id_u][id_v]=result
    tabelaSimilaridade[id_v][id_u]=result
    return result


def user_based(user_id, item_id,data,tabelaSimilaridade):
    Nui = userData[userData['user id'].isin(
        data[data['item id'] == item_id]['user id'])]['user id']
    # print(""+str(user_id)+" "+str(item_id))
    # print(len(Nui))
    if(len(Nui)==0):
        return None
    
    # parcial1 = sum(Nui.apply(lambda x:
    #                          ((cossine_similarity(data[data['user id'] == user_id],
    #                                              data[data['user id']
    #                                                   == x],tabelaSimilaridade
    #                                              )if math.isnan(tabelaSimilaridade[user_id][x])else  tabelaSimilaridade[user_id][x])*(data[(
    #                                             data['user id'] == x) & (data['item id'] == item_id)
    #                                              ]['rating'].reset_index(drop=True)), axis=1

    #                           ).rename(columns={0: 'rating'})['rating'])
    parcial1 = sum(Nui.apply(lambda x:
                             ((cossine_similarity(data[data['user id'] == user_id],
                                                 data[data['user id']
                                                      == x],tabelaSimilaridade
                                                  )if math.isnan(tabelaSimilaridade[user_id][x])else tabelaSimilaridade[user_id][x])
                                                  *data[(
                                                     data['user id'] == x) & (data['item id'] == item_id)
                             ]['rating'].reset_index(drop=True))

                             ).rename(columns={0: 'rating'})['rating'])
    print(parcial1)          
    # parcial1 = sum(Nui.apply(lambda x:
    #                          (
    #                             print("foi")
    #                              if math.isnan(tabelaSimilaridade[user_id][x['user id']]) else print("Nao foi")
    #                              )
    #                          ))
    parcial2 = sum(Nui.apply(lambda x:
                             (abs(cossine_similarity(data[data['user id'] == user_id],
                                                     data[data['user id']
                                                          == x],
                                                          tabelaSimilaridade
                                                     )) if math.isnan( tabelaSimilaridade[user_id][x])  else abs(tabelaSimilaridade[user_id][x]) )
                             ))
    return parcial1/parcial2


In [82]:
similaridade = pd.DataFrame(index=userData['user id'],columns=userData['user id'])
for id in userData['user id']:
    similaridade[id][id]=1
similaridade

user id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user id,,,,,,,,,,,,,,,,,,,,,
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN


In [91]:
test=dataOriginal.drop(columns='timestamp')
train,test1=train_test_split(test,train_size=0.1)
x_test=train.drop('rating',axis=1)
y_test=train['rating']
x_test

,user id,item id
60244,840,191
60143,798,155
61972,422,867
54895,732,882
69851,674,1197
...,...,...
25229,378,7
89417,455,385
13295,158,82
59488,64,751


In [92]:
y_test_predict= x_test.apply(lambda x:
           user_based(x['user id'],x['item id'],test,similaridade) ,axis=1)
y_test_predict

1102.4234891428341
283.300578330573
22.641095435708202
104.82648457069901
153.22460913501146
1667.1559757631567
51.45958170033644
494.05935648713773
992.9899691672487
765.89186810816
312.5687735899056
711.373673286629
1181.1855159999798
747.1375736997088
742.3563372068916
233.46833146044497
125.49416519639429
874.7724689082754
28.459315635429093
490.1591963947522
204.0822591438551
208.1928477060044
768.9773618480136
52.95182403708819
184.3015940338619
136.39220180095938
1475.1733600491166
120.24020524656545
356.17950277301406
1179.3011917289969
354.4334403332039
1040.017474357599
635.3398757151299
210.11346518175958
31.898049749006994
675.5724155820121
507.92285144624265
849.1327317866222
353.6739146937485
206.87634663724398
473.29801811552426
64.00658570014306
659.1669697257183
1095.9056608086707
202.5924852117062
70.738408433569
508.91176777275194
1029.623545516068
698.8180645453231
390.02891780635883
81.37288823766609
240.93163065055305
1687.2199812471044
106.95784011875152
630.3075

60244    4.168652
60143    3.105854
61972    3.974657
54895    3.326011
69851    3.560251
           ...   
25229    3.802869
89417    3.570397
13295    3.725213
59488    3.424336
34556    4.468701
Length: 10000, dtype: float64

In [97]:
y_predict=y_test_predict.replace(np.nan,0)
y_predict[y_predict==0]

Series([], dtype: float64)

In [98]:
y_test

60244    4
60143    3
61972    3
54895    5
69851    3
        ..
25229    4
89417    3
13295    5
59488    2
34556    2
Name: rating, Length: 10000, dtype: int64

In [99]:
similaridade

user id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user id,,,,,,,,,,,,,,,,,,,,,
1,1,0.960582,0.857075,0.919264,0.932614,0.953845,0.943945,0.977505,0.976404,0.968304,...,0.942682,0.875338,0.965889,0.91633,0.910709,0.944993,0.947752,0.970836,0.923505,0.929744
2,0.960582,1,0.935601,0.946756,0.984803,0.955547,0.967087,0.958659,NaN,0.982614,...,0.950765,0.957073,0.9674,0.966063,0.946455,0.974901,0.93611,0.940225,0.948493,NaN
3,0.857075,0.935601,1,0.919528,NaN,NaN,0.879543,0.89589,NaN,0.902444,...,NaN,NaN,0.896328,0.840511,NaN,0.993884,0.852826,0.973223,NaN,NaN
4,0.919264,0.946756,0.919528,1,0.994692,0.936996,0.882332,0.976533,NaN,0.969117,...,NaN,NaN,0.952164,0.895669,0.958333,1.0,0.96926,0.998538,0.980173,0.976417
5,0.932614,0.984803,NaN,0.994692,1,0.933627,0.908163,0.941228,0.880919,0.940114,...,0.920066,0.943281,0.941549,NaN,0.903752,0.943701,0.918769,0.975643,0.946235,0.909228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.944993,0.974901,0.993884,1.0,0.943701,0.928228,0.957014,NaN,NaN,NaN,...,0.951155,0.962738,0.975279,0.921936,0.944249,1,NaN,0.965729,0.980407,0.954349
940,0.947752,0.93611,0.852826,0.96926,0.918769,0.944746,0.941044,0.95436,0.982175,0.967073,...,0.936986,0.95277,0.918801,0.880023,0.926234,NaN,1,0.967752,0.960041,0.962575
941,0.970836,0.940225,0.973223,0.998538,0.975643,0.945658,0.9374,0.931134,NaN,0.986317,...,0.943701,0.962572,0.974237,NaN,0.934363,0.965729,0.967752,1,NaN,0.960754


In [100]:
mean_absolute_error(y_test,y_predict)

0.7905855192900323